In [ ]:
 # we will be storing the rating data as its own 
    # table in SQL, so we'll need to convert it to a datetime data type. 
    #There are other epochs in use, but the Unix epoch is by far the most widespread.
    # Specify in to_datetime() that the origin is 'unix' and the time unit is seconds.
    # pd.to_datetime(ratings['timestamp'], unit='s')
    
    # assign it to the timestamp column.
    # ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
    
    # use a groupby on the "movieId" and "rating" columns and take the 
    # count for each group.
    # Used reference code from M_Fullerton "ETL_clean_kaggle_data.ipynb.
    # rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count()   
    
    # Rename the "userId" column to "count."
       # pivot this data so that movieId is the index, the columns will be 
    # all the rating values, and the rows will be the counts for each 
    # rating value.
    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count()
    
                .rename({'userId':'count'}, axis=1) \
                .pivot(index='movieId',columns='rating', values='count')
    
    #  rename the columns so they're easier to understand. We'll prepend 
    # rating_ to each column with a list comprehension
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]    
    
    # Merge the ratings DataFrame with the movies_df DataFrame, name the new DataFrame movies_with_ratings_df
    # Used reference code from M_Fullerton "ETL_clean_kaggle_data.ipynb." Modified pd.merge function using "right" and "how"
    # parameters. We need to use a left merge, since we want to keep everything in movies_df:
    movies_with_ratings_df = pd.merge(movies_df, ratings, left_on='kaggle_id', right_index=True, how='left')
    
    # Not every movie got a rating for each rating level, there will be  missing values instead of zeros. 
    # We have to fill those in ourselves, 
    movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)
    
    return wiki_movies_df, movies_with_ratings_df, movies_df